In [1]:
import openslide
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import tqdm

In [2]:
slides = glob.glob('/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/*.svs')
slides

['/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201121868_HE_3.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201122039_HE_3.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201122061_HE_1.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201122287_HE_3.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201122320_HE_3.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201122324_HE_null.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201122326_HE_4.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201123052_HE_2.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201124026_HE_4.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201124041_HE_1.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201124487_HE_3.svs',
 '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201125276_HE_null.svs',
 '/home/lhm/mnt/medica

In [3]:
import concurrent.futures
from tqdm import tqdm
from multiprocessing import Pool, Pipe, freeze_support

#=============================================================#
# 接口                                                        #
#-------------------------------------------------------------#
#   multi_process_exec 多进程执行                             #
#   multi_thread_exec  多线程执行                             #
#-------------------------------------------------------------#
# 参数：                                                      #
#   f         (function): 批量执行的函数                      #
#   args_mat  (list)    : 批量执行的参数                      #
#   pool_size (int)     : 进程/线程池的大小                   #
#   desc      (str)     : 进度条的描述文字                    #
#-------------------------------------------------------------#
# 例子：                                                      #
# >>> def Pow(a,n):        ← 定义一个函数（可以有多个参数）   #
# ...     return a**n                                         #
# >>>                                                         #
# >>> args_mat=[[2,1],     ← 批量计算 Pow(2,1)                #
# ...           [2,2],                Pow(2,2)                #
# ...           [2,3],                Pow(2,3)                #
# ...           [2,4],                Pow(2,4)                #
# ...           [2,5],                Pow(2,5)                #
# ...           [2,6]]                Pow(2,6)                #
# >>>                                                         #
# >>> results=multi_thread_exec(Pow,args_mat,desc='计算中')   #
# 计算中: 100%|█████████████| 6/6 [00:00<00:00, 20610.83it/s] #
# >>>                                                         #
# >>> print(results)                                          #
# [2, 4, 8, 16, 32, 64]                                       #
#-------------------------------------------------------------#

ToBatch = lambda arr, size: [arr[i * size:(i + 1) * size] for i in range((size - 1 + len(arr)) // size)]


def batch_exec(f, args_batch, w):
    results = []
    for i, args in enumerate(args_batch):
        try:
            if isinstance(args, (list, tuple, dict)):
                ans = f(*args)
            else:
                ans = f(args)
            results.append(ans)
        except Exception as e:
            print(e)
            results.append(None)
        w.send(1)
    return results


def multi_process_exec(f, args_mat, pool_size=5, desc=None):
    if len(args_mat) == 0: return []
    batch_size = max(1, int(len(args_mat) / 4 / pool_size))
    results = []
    args_batches = ToBatch(args_mat, batch_size)
    with tqdm(total=len(args_mat), desc=desc) as pbar:
        with Pool(processes=pool_size) as pool:
            r, w = Pipe(duplex=False)
            pool_rets = []
            for i, args_batch in enumerate(args_batches):
                pool_rets.append(pool.apply_async(batch_exec, (f, args_batch, w)))
            cnt = 0
            while cnt < len(args_mat):
                try:
                    msg = r.recv()
                    pbar.update(1)
                    cnt += 1
                except EOFError:
                    print('EOFError')
                    break
            for ret in pool_rets:
                for r in ret.get():
                    results.append(r)
    return results


def multi_thread_exec(f, args_mat, pool_size=5, desc=None):
    if len(args_mat) == 0: return []
    results = [None for _ in range(len(args_mat))]
    with tqdm(total=len(args_mat), desc=desc) as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=pool_size) as executor:
            futures = {executor.submit(f, *args): i for i, args in enumerate(args_mat)}
            for future in concurrent.futures.as_completed(futures):
                i = futures[future]
                ret = future.result()
                results[i] = ret
                pbar.update(1)
    return results

In [4]:
import time
import xml.dom.minidom as dom
from itertools import permutations

import torch
import torch.nn as nn


def read_points_from_xml(liver_name, scale=1,
                         xml_path='/mnt/datasets/medical/yxt/CAMELYON16/training/lesion_annotations/', dataset=''):
    if dataset == '':
        if 'HCC' in xml_path:
            dataset = 'HCC'
        elif 'CAMELYON16' in xml_path:
            dataset = 'CAMELYON16'
        elif 'TCGA' in xml_path:
            dataset = 'TCGA'

    if dataset == 'HCC':
        xml = dom.parse(xml_path + liver_name)
        tag_name = 'Annotation'
        point_name = 'Coordinate'
        x_name = 'X'
        y_name = 'Y'
    elif dataset == 'CAMELYON16':
        xml = dom.parse(xml_path + liver_name)
        tag_name = 'Annotation'
        point_name = 'Coordinate'
        x_name = 'X'
        y_name = 'Y'
    elif dataset == 'TCGA':
        xml = dom.parse(xml_path + liver_name)
        tag_name = 'Region'
        point_name = 'Vertex'
        x_name = 'X'
        y_name = 'Y'
    anno_list = xml.documentElement.getElementsByTagName(tag_name)
    polygons = []
    for anno in anno_list:
        polygons.append([])
        for point in anno.getElementsByTagName(point_name):
            x = int(float(point.getAttribute(x_name)))
            y = int(float(point.getAttribute(y_name)))
            polygons[-1].append([x // scale, y // scale])
    return polygons

In [ ]:
def Pow(a, n):
    return a ** n


args_mat = [[2, 4], [3, 2], [4, 5]]
results = multi_process_exec(Pow, args_mat, desc='计算中')
print(results)

In [ ]:
def save_thumb(slide):
    code = slide.split('/')[-1].split('.')[-2]
    s = openslide.open_slide(slide)
    if (s.level_count < 3):
        return
    size = s.level_dimensions[2]
    img = np.array(s.read_region((0, 0), 2, size).convert('RGB'))
    samples = np.array(s.level_downsamples, dtype=np.int32)
    if samples[2] == 16:
        img_reshape = cv2.resize(img, (size[0] // 4, size[1] // 4))
    elif samples[2] == 32:
        img_reshape = cv2.resize(img, (size[0] // 2, size[1] // 2))
    cv2.imwrite(f"/mnt/datasets/medical/lhm/liverWSI/gnn_1/{code}.png", img_reshape)


multi_process_exec(save_thumb, slides, 64, desc='processing')

In [ ]:
name = "201202725_HE_4"
img = test1_reshape
polygons = read_points_from_xml(liver_name=f'{name}.xml',
                                xml_path="/mnt/datasets/medical/yxt/liverWSI/Hepatoma_2_5_years_800/", scale=64,
                                dataset='HCC')
mask = np.zeros(img.shape[:2], dtype=np.int8)
for index, p in enumerate(polygons):
    polygons[index] = np.array(p, dtype=np.int32)
cv2.polylines(mask, polygons, True, 255)
cv2.fillPoly(mask, polygons, 255)
plt.subplot(1, 2, 1)
plt.imshow(mask)
plt.subplot(1, 2, 2)
plt.imshow(img)
plt.show()

In [ ]:
# def convert_numpy_img_to_superpixel_graph(img, polygons, code, slic_kwargs={}):
#     height = img.shape[0]
#     width = img.shape[1]
#     n = img.shape[0] * img.shape[1] // 1024
#     segments = slic(img, n_segments=n, compactness=20, slic_zero=True, start_label=0, **slic_kwargs)
#     np.save(f"/mnt/datasets/medical/lhm/liverWSI/gnn_1_seg/{code}.npy", segments)
#     num_of_nodes = np.max(segments) + 1
#     nodes = {node: {"rgb_list": []} for node in range(num_of_nodes)}
#     # get rgb values and positions
#     for y in range(height):
#         for x in range(width):
#             node = segments[y, x]
#             rgb = img[y, x, :]
#             nodes[node]["rgb_list"].append(rgb)
#     for node in nodes:
#         nodes[node]["rgb_list"] = np.stack(nodes[node]["rgb_list"])
#     G = nx.Graph()
#     # compute node positions
#     segments_ids = np.unique(segments)
#     pos = np.array([np.mean(np.nonzero(segments == i), axis=1) for i in segments_ids])
#     pos = pos.astype(int)
#     #pos[0]为height_y pos[1]为width_x
#     for node in nodes:
#         feature = nodes[node]['rgb_list'][:1024]
#         if len(feature) < 1024:
#             feature = np.pad(feature, ((0, 1024 - len(feature)), (0, 0)), 'edge')
#         for p in polygons:
#             p = np.array(p, dtype=np.int32)
#             label = cv2.pointPolygonTest(p, (int(pos[node][1] * 8), int(pos[node][0] * 8)), True) > 0
#         G.add_node(node, features=feature, label=(2 if label else 1))
#     # add edges
#     vs_right = np.vstack([segments[:, :-1].ravel(), segments[:, 1:].ravel()])
#     vs_below = np.vstack([segments[:-1, :].ravel(), segments[1:, :].ravel()])
#     bneighbors = np.unique(np.hstack([vs_right, vs_below]), axis=1)
#     for i in range(bneighbors.shape[1]):
#         if bneighbors[0, i] != bneighbors[1, i]:
#             G.add_edge(bneighbors[0, i], bneighbors[1, i])
#     # add self loops
#     for node in nodes:
#         G.add_edge(node, node)
# 
#     # get edge_index
#     m = len(G.edges)
#     edge_index = np.zeros([2 * m, 2]).astype(np.int64)
#     for e, (s, t) in enumerate(G.edges):
#         edge_index[e, 0] = s
#         edge_index[e, 1] = t
#         edge_index[m + e, 0] = t
#         edge_index[m + e, 1] = s
# 
#     # get features
#     # num_of_nodes = len(nodes)
#     num_of_features = 1024
#     x = np.zeros([num_of_nodes, num_of_features, 3]).astype(np.float32)
#     y = np.zeros(num_of_nodes).astype(np.float32)
#     coord = dict({})
#     for node in G.nodes:
#         x[node] = G.nodes[node]["features"]
#         y[node] = G.nodes[node]["label"]
#     return x, edge_index, pos, y

In [6]:
import multiprocessing
import os
import time
from itertools import repeat

import networkx as nx
import numpy as np
import torch
from skimage.segmentation import slic, mark_boundaries
from torch_geometric.data import Data

# tqdm for progress bar
from tqdm.auto import tqdm


def convert_numpy_img_to_superpixel_graph(img, polygons, code, slic_kwargs={}):
    height = img.shape[0]
    width = img.shape[1]
    n = 1024
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 255 - 15])  # 假设threshold是一个你选择的值
    upper_white = np.array([180, 255, 255])
    mask = 255 - cv2.inRange(hsv_image, lower_white, upper_white)
    kernel = np.ones((10, 10), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = mask / 255
    segments = slic(img, n_segments=n, slic_zero=True, compactness=10, start_label=0,
                    enforce_connectivity=True, convert2lab=True, sigma=0.7,
                    mask=mask, **slic_kwargs)

    np.save(f"/home/lhm/mnt/medical/lhm/liverWSI/gnn_1_seg/{code}.npy", segments)
    num_of_nodes = np.max(segments) + 1
    nodes = {node: {"rgb_list": [], "r": [], "g": [], "b": [], } for node in range(num_of_nodes)}
    # get rgb values and positions
    for y in range(height):
        for x in range(width):
            node = segments[y, x]
            if node < 0:
                continue
            rgb = img[y, x, :]
            nodes[node]["r"].append(rgb[2])
            nodes[node]["g"].append(rgb[1])
            nodes[node]["b"].append(rgb[0])
    for node in nodes:
        r_bin = np.bincount(nodes[node]["r"])
        r_bin = np.pad(r_bin, (0, 256 - len(r_bin)), 'constant', constant_values=(0, 0))
        g_bin = np.bincount(nodes[node]["g"])
        g_bin = np.pad(g_bin, (0, 256 - len(g_bin)), 'constant', constant_values=(0, 0))
        b_bin = np.bincount(nodes[node]["b"])
        b_bin = np.pad(b_bin, (0, 256 - len(b_bin)), 'constant', constant_values=(0, 0))
        nodes[node]["rgb_list"] = np.stack([r_bin, g_bin, b_bin]).ravel()
    G = nx.Graph()
    # compute node positions
    segments_ids = np.unique(segments)
    pos = np.array([np.mean(np.nonzero(segments == i), axis=1) for i in segments_ids])
    pos = pos.astype(int)
    #pos[0]为height_y pos[1]为width_x
    for node in nodes:
        feature = nodes[node]['rgb_list']
        label = 0
        for p in polygons:
            p = np.array(p, dtype=np.int32)
            label = cv2.pointPolygonTest(p, (int(pos[node][1]), int(pos[node][0])), True) > 0
        G.add_node(node, features=feature, label=label)
    # add edges
    vs_right = np.vstack([segments[:, :-1].ravel(), segments[:, 1:].ravel()])
    vs_below = np.vstack([segments[:-1, :].ravel(), segments[1:, :].ravel()])
    bneighbors = np.unique(np.hstack([vs_right, vs_below]), axis=1)
    for i in range(bneighbors.shape[1]):
        if bneighbors[0, i] == -1 or bneighbors[1, i] == -1:
            continue
        if bneighbors[0, i] != bneighbors[1, i]:
            G.add_edge(bneighbors[0, i], bneighbors[1, i])
    # add self loops
    for node in nodes:
        G.add_edge(node, node)

    # get edge_index
    m = len(G.edges)
    edge_index = np.zeros([2 * m, 2]).astype(np.int64)
    for e, (s, t) in enumerate(G.edges):
        edge_index[e, 0] = s
        edge_index[e, 1] = t
        edge_index[m + e, 0] = t
        edge_index[m + e, 1] = s

    # get features
    # num_of_nodes = len(nodes)
    num_of_features = 768
    x = np.zeros([num_of_nodes, num_of_features]).astype(np.float32)
    y = np.zeros(num_of_nodes).astype(np.float32)
    for node in G.nodes:
        x[node] = G.nodes[node]["features"]
        y[node] = G.nodes[node]["label"]
    return x, edge_index, pos, y

In [ ]:
# img = cv2.imread('/nfs3/lhm/HCC/gnn_1/201357767_HE_5.png')
# name = i.split('/')[-1].split('.')[0]
# polygons = read_points_from_xml(liver_name= f'{name}.xml',scale=8,xml_path='/medical-data/yxt/liverWSI/Hepatoma_2_5_years_800/',dataset='HCC')
# x, edge_index, pos, y = convert_numpy_img_to_superpixel_graph(img,polygons)

In [7]:
def process_img(i):
    img = cv2.imread(i)
    name = i.split('/')[-1].split('.')[0]
    polygons = read_points_from_xml(liver_name=f'{name}.xml', scale=64,
                                    xml_path='/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/',
                                    dataset='HCC')
    x, edge_index, pos, y = convert_numpy_img_to_superpixel_graph(img, polygons, name)
    res = dict({})
    res['x'] = x
    res['edge'] = edge_index
    res['pos'] = pos
    res['y'] = y
    res['code'] = name
    np.save(f'/home/lhm/mnt/medical/lhm/liverWSI/gnn_1/{name}.npy', res, allow_pickle=True)


process_img('/home/lhm/mnt/medical/lhm/liverWSI/gnn_1/201122061_HE_1.png')

In [8]:
args_mat = glob.glob('/home/lhm/mnt/medical/lhm/liverWSI/gnn_1/*.png')
multi_process_exec(process_img, args_mat, 24, desc='processing')

processing:   0%|          | 0/812 [00:00<?, ?it/s]

/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarni

[Errno 2] No such file or directory: '/home/lhm/mnt/medical/yxt/liverWSI/Hepatoma_2_5_years_800/201355521_HE_1.xml'


/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/home/lhm/miniconda3/envs/pytorch/lib/python3.9/site-packages/scipy/cluster/vq.py:602: UserWarni

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [31]:
a = np.load('/home/lhm/mnt/medical/lhm/liverWSI/gnn_1/201122061_HE_1.npy',allow_pickle=True).item()
a['x'].shape

(1024, 768)